In [1]:
# !pip install --upgrade tensorflow --quiet
# !pip install keras_tuner --quiet
# !pip install tensorflow-io --quiet
# # Google colab modules
# from google.colab import drive
import sys, importlib

# # Mount drive
# drive.mount('/content/gdrive', force_remount=True)
ROOT_PATH = './'
# sys.path.append(ROOT_PATH)

import coremlv2 as core
core._init_ml()
# core._init_models()
# core.os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Reload coreml
importlib.reload(core)
import keras_tuner as kt

In [2]:
# Limiting GPU memory growth
gpus = core.tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            core.tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = core.tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs, ", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs,  1 Logical GPUs


### Manually retrain model

In [3]:
model_base_id = '327'
version = '3'
model_no = model_base_id
constituent_limits = 0.025
id_constituent = 1
min_vid_constituents = 0.5
epochs = 5
iter_id = f'model-{model_no}-{version}_constituent_limits-{constituent_limits}_id_constituent-{id_constituent}_min_vid_constituents-{min_vid_constituents}_epochs-{epochs}'
ROOT_PATH='./'
DB_ROOT_PATH='J:\#PROJECT\idx'
db_ver = '8'
batch_size = 64
shuffle_buffer_size = 1
generator=True

In [4]:
# Train: `slice_from_beginning`=True
train_gen = core.load_dataset_wsd(slice_from_beginning=True, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=constituent_limits, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)
# Validation: `slice_from_beginning`=False. constituent_limits in validation is always 0 (focus on idx performance progression only)
validation_gen = core.load_dataset_wsd(slice_from_beginning=False, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=0, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)

# Load model backbone
model = core.model_switcher_preloaded(model_no, version=version)
history = model.fit(train_gen, validation_data=validation_gen, epochs=epochs, verbose=1)

save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{iter_id}/')
model.save_weights(f'{save_path}/weights/checkpoint')
model.save(f'{save_path}/model')

history.history['batch_size'] = batch_size
history.history['shuffle_buffer_size'] = shuffle_buffer_size

with open(f'{save_path}/history.json', 'w') as f:
    core.json.dump(history.history, f)

Total constituents: 1149
Total constituents: 422
Epoch 1/5
19515/19515 [==============================] - 2102s 107ms/step - loss: 0.6646 - accuracy: 0.6034 - val_loss: 0.7395 - val_accuracy: 0.5617
Epoch 2/5
19515/19515 [==============================] - 1996s 102ms/step - loss: 0.6165 - accuracy: 0.6564 - val_loss: 0.9229 - val_accuracy: 0.5365
Epoch 3/5
19515/19515 [==============================] - 1440s 74ms/step - loss: 0.5655 - accuracy: 0.7037 - val_loss: 1.0950 - val_accuracy: 0.5045
Epoch 4/5
19515/19515 [==============================] - 2122s 109ms/step - loss: 0.5348 - accuracy: 0.7280 - val_loss: 1.1303 - val_accuracy: 0.5009
Epoch 5/5
19515/19515 [==============================] - 1403s 72ms/step - loss: 0.5148 - accuracy: 0.7427 - val_loss: 1.2824 - val_accuracy: 0.4905


In [3]:
model_base_id = '327'
version = '3'
model_no = model_base_id
constituent_limits = 0.025
id_constituent = 1
min_vid_constituents = 0.7
epochs = 10
batch_size = 512
iter_id = f'model-{model_no}-{version}_constituentlimits-{constituent_limits}_idconstituent-{id_constituent}_minvidconstituents-{min_vid_constituents}_epochs-{epochs}_batchsize-{batch_size}_2'
ROOT_PATH='./'
DB_ROOT_PATH='J:\#PROJECT\idx'
db_ver = '8'

shuffle_buffer_size = 1
generator=True

In [4]:
# Train: `slice_from_beginning`=True
train_gen = core.load_dataset_wsd(slice_from_beginning=True, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=constituent_limits, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no, min_vid_constituents=min_vid_constituents)
# Validation: `slice_from_beginning`=False. constituent_limits in validation is always 0 (focus on idx performance progression only)
validation_gen = core.load_dataset_wsd(slice_from_beginning=False, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=0, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no, min_vid_constituents=min_vid_constituents)

# Load model backbone
model = core.model_switcher_preloaded(model_no, version=version)
history = model.fit(train_gen, validation_data=validation_gen, epochs=epochs, verbose=1)

save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{iter_id}/')
model.save_weights(f'{save_path}/weights/checkpoint')
model.save(f'{save_path}/model')

with open(f'{save_path}/history.json', 'w') as f:
    core.json.dump(history.history, f)

Total constituents: 1531
Total constituents: 422
Epoch 1/10
3892/3892 [==============================] - 1958s 501ms/step - loss: 0.6803 - accuracy: 0.5843 - val_loss: 0.6730 - val_accuracy: 0.5826
Epoch 2/10
3892/3892 [==============================] - 1982s 509ms/step - loss: 0.6556 - accuracy: 0.6084 - val_loss: 0.6762 - val_accuracy: 0.5743
Epoch 3/10
3892/3892 [==============================] - 1939s 498ms/step - loss: 0.6488 - accuracy: 0.6148 - val_loss: 0.6756 - val_accuracy: 0.5802
Epoch 4/10
3892/3892 [==============================] - 1894s 487ms/step - loss: 0.6424 - accuracy: 0.6209 - val_loss: 0.6843 - val_accuracy: 0.5713
Epoch 5/10
3892/3892 [==============================] - 1780s 457ms/step - loss: 0.6365 - accuracy: 0.6263 - val_loss: 0.6882 - val_accuracy: 0.5714
Epoch 6/10
3892/3892 [==============================] - 1717s 441ms/step - loss: 0.6305 - accuracy: 0.6319 - val_loss: 0.7007 - val_accuracy: 0.5659
Epoch 7/10
3892/3892 [==============================] - 1

In [3]:
model_base_id = '327'
version = '5'
model_no = model_base_id
constituent_limits = 0.25
id_constituent = 1
min_vid_constituents = 0.2
epochs = 1
batch_size = 512
iter_id = f'model-{model_no}-{version}_constituentlimits-{constituent_limits}_idconstituent-{id_constituent}_minvidconstituents-{min_vid_constituents}_epochs-{epochs}_batchsize-{batch_size}_2'
ROOT_PATH='./'
DB_ROOT_PATH='J:\#PROJECT\idx'
db_ver = '8'

shuffle_buffer_size = 1
generator=True

In [ ]:
# Train: `slice_from_beginning`=True
train_gen = core.load_dataset_wsd(slice_from_beginning=True, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=constituent_limits, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)
# Validation: `slice_from_beginning`=False. constituent_limits in validation is always 0 (focus on idx performance progression only)
validation_gen = core.load_dataset_wsd(slice_from_beginning=False, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=0, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)

# Load model backbone
model = core.model_switcher_preloaded(model_no, version=version)
if type(train_gen) == list:
    for epoch in range(epochs):
        for i, tg in enumerate(train_gen):
            print(f'Epoch {epoch+1}-{i+1}/{epochs}')
            history = model.fit(tg, validation_data=validation_gen, epochs=1, verbose=1)
elif type(train_gen) != list:
    history = model.fit(train_gen, validation_data=validation_gen, epochs=epochs, verbose=1)

save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{iter_id}/')
model.save_weights(f'{save_path}/weights/checkpoint')
model.save(f'{save_path}/model')

with open(f'{save_path}/history.json', 'w') as f:
    core.json.dump(history.history, f)

Total constituents: 9006
Total constituents: 422
Epoch 1-1/1
5036/5036 [==============================] - 1939s 384ms/step - loss: 0.7248 - accuracy: 0.5221 - val_loss: 0.6996 - val_accuracy: 0.5409
Epoch 1-2/1
5164/5164 [==============================] - 2002s 388ms/step - loss: 0.6909 - accuracy: 0.5460 - val_loss: 0.6836 - val_accuracy: 0.5615
Epoch 1-3/1
4974/4974 [==============================] - 1924s 387ms/step - loss: 0.6790 - accuracy: 0.5676 - val_loss: 0.6757 - val_accuracy: 0.5772
Epoch 1-4/1
5283/5283 [==============================] - 2331s 441ms/step - loss: 0.6725 - accuracy: 0.5806 - val_loss: 0.6701 - val_accuracy: 0.5870
Epoch 1-5/1
5029/5029 [==============================] - 2198s 437ms/step - loss: 0.6681 - accuracy: 0.5889 - val_loss: 0.6670 - val_accuracy: 0.5924
Epoch 1-6/1
    809/Unknown - 334s 412ms/step - loss: 0.6642 - accuracy: 0.5944

In [3]:
model_base_id = '327'
version = '5'
model_no = model_base_id
constituent_limits = 0.25
id_constituent = 1
min_vid_constituents = 0.2
epochs = 5
batch_size = 1024
iter_id = f'model-{model_no}-{version}_constituentlimits-{constituent_limits}_idconstituent-{id_constituent}_minvidconstituents-{min_vid_constituents}_epochs-{epochs}_batchsize-{batch_size}_2'
ROOT_PATH='./'
DB_ROOT_PATH='J:\#PROJECT\idx'
db_ver = '8'

shuffle_buffer_size = 1
generator=True

In [ ]:
# Train: `slice_from_beginning`=True
train_gen = core.load_dataset_wsd(slice_from_beginning=True, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=constituent_limits, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)
# Validation: `slice_from_beginning`=False. constituent_limits in validation is always 0 (focus on idx performance progression only)
validation_gen = core.load_dataset_wsd(slice_from_beginning=False, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=0, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)

# Load model backbone
model = core.model_switcher_preloaded(model_no, version=version)
if type(train_gen) == list:
    for epoch in range(epochs):
        for i, tg in enumerate(train_gen):
            print(f'Epoch {epoch+1}-{i+1}/{epochs}')
            history = model.fit(tg, validation_data=validation_gen, epochs=1, verbose=1)
elif type(train_gen) != list:
    history = model.fit(train_gen, validation_data=validation_gen, epochs=epochs, verbose=1)

save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{iter_id}/')
model.save_weights(f'{save_path}/weights/checkpoint')
model.save(f'{save_path}/model')

with open(f'{save_path}/history.json', 'w') as f:
    core.json.dump(history.history, f)

Total constituents: 9006
Total constituents: 422
Epoch 1-1/5
2510/2510 [==============================] - 1935s 768ms/step - loss: 0.8095 - accuracy: 0.5124 - val_loss: 0.7287 - val_accuracy: 0.5333
Epoch 1-2/5
2589/2589 [==============================] - 2001s 773ms/step - loss: 0.7166 - accuracy: 0.5312 - val_loss: 0.6951 - val_accuracy: 0.5491
Epoch 1-3/5
2482/2482 [==============================] - 1905s 767ms/step - loss: 0.6920 - accuracy: 0.5518 - val_loss: 0.6819 - val_accuracy: 0.5647
Epoch 1-4/5
2625/2625 [==============================] - 2087s 795ms/step - loss: 0.6815 - accuracy: 0.5663 - val_loss: 0.6737 - val_accuracy: 0.5778
Epoch 1-5/5
2501/2501 [==============================] - 1923s 769ms/step - loss: 0.6740 - accuracy: 0.5796 - val_loss: 0.6694 - val_accuracy: 0.5865
Epoch 1-6/5
2554/2554 [==============================] - 1935s 757ms/step - loss: 0.6698 - accuracy: 0.5859 - val_loss: 0.6662 - val_accuracy: 0.5923
Epoch 1-7/5
2508/2508 [============================

In [3]:
model_base_id = '327'
version = '5'
model_no = model_base_id
constituent_limits = 0.5
id_constituent = 1
min_vid_constituents = 0.3
epochs = 10
batch_size = 1024
iter_id = f'model-{model_no}-{version}_constituentlimits-{constituent_limits}_idconstituent-{id_constituent}_minvidconstituents-{min_vid_constituents}_epochs-{epochs}_batchsize-{batch_size}_2'
ROOT_PATH='./'
DB_ROOT_PATH='J:\#PROJECT\idx'
db_ver = '8'

shuffle_buffer_size = 1
generator=True

In [ ]:
# Train: `slice_from_beginning`=True
train_gen = core.load_dataset_wsd(slice_from_beginning=True, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=constituent_limits, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no, min_vid_constituents=min_vid_constituents)
# Validation: `slice_from_beginning`=False. constituent_limits in validation is always 0 (focus on idx performance progression only)
validation_gen = core.load_dataset_wsd(slice_from_beginning=False, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=0, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no, min_vid_constituents=min_vid_constituents)

# Load model backbone
model = core.model_switcher_preloaded(model_no, version=version)
if type(train_gen) == list:
    for epoch in range(epochs):
        for i, tg in enumerate(train_gen):
            print(f'Epoch {epoch+1}-{i+1}/{epochs}')
            history = model.fit(tg, validation_data=validation_gen, epochs=1, verbose=1)
elif type(train_gen) != list:
    history = model.fit(train_gen, validation_data=validation_gen, epochs=epochs, verbose=1)

save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{iter_id}/')
model.save_weights(f'{save_path}/weights/checkpoint')
model.save(f'{save_path}/model')

with open(f'{save_path}/history.json', 'w') as f:
    core.json.dump(history.history, f)

Total constituents: 19482
Total constituents: 422
Epoch 1-1/10
2416/2416 [==============================] - 1813s 748ms/step - loss: 0.8080 - accuracy: 0.5154 - val_loss: 0.7305 - val_accuracy: 0.5185
Epoch 1-2/10
2404/2404 [==============================] - 1779s 740ms/step - loss: 0.7073 - accuracy: 0.5324 - val_loss: 0.7005 - val_accuracy: 0.5359
Epoch 1-3/10
2424/2424 [==============================] - 1800s 742ms/step - loss: 0.6903 - accuracy: 0.5502 - val_loss: 0.6875 - val_accuracy: 0.5539
Epoch 1-4/10
    820/Unknown - 581s 706ms/step - loss: 0.6850 - accuracy: 0.5584